In [1]:
Je réfléchis à utiliser les données versionnées des GOP plutôt que des PTF pour notre système de flagging quotidien, étant donné la qualité et la structure des données que nous avons à ce niveau. Tu penses que cela pourrait être plus pertinent pour ton analyse quotidienne ?

  key1  key2 value1 value2
0    A     1    foo    one
1    B     2    bar    two
2    C     3    baz  three
3    D     4    qux    NaN
